In [6]:
import os
import pandas as pd
result_df = pd.read_csv('./work_dir/output-123.csv')
# result_dro_df = pd.read_csv('./work_dir/dro_metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_dice', 'rim_dice', 'cup_iou', 'rim_iou']
# class_list = ['cup_metric', 'rim_metric']
attribute_list = ['gender', 'race', 'ethnicity']

from load_data import FairEvaluator

fair_evaluator = FairEvaluator(result_df, att_df, class_list)
fairness_metrics = fair_evaluator('race')

# 将公平性评估数据写入 CSV 文件
output_csv_path = './work_dir/fairness-123.csv'  # 指定输出文件路径

# 将字典转换为 DataFrame
fairness_df = pd.DataFrame.from_dict(fairness_metrics, orient='index')

# 将 DataFrame 写入 CSV 文件
fairness_df.to_csv(output_csv_path, index=True)

print(f"公平性评估数据已保存到: {output_csv_path}")

公平性评估数据已保存到: ./work_dir/fairness-123.csv


In [3]:
fair_evaluator('ethnicity')


{'cup_dice': {'overall': 0.9122171179652214,
  0: 0.91182121857427,
  1: 0.916595995426178,
  'es_cup_dice': 0.9078821831329386},
 'rim_dice': {'overall': 0.8582400793507695,
  0: 0.8578759302416913,
  1: 0.857079970339934,
  'es_rim_dice': 0.8569338909093217},
 'cup_iou': {'overall': 0.8415818624049425,
  0: 0.8409605296892171,
  1: 0.8491414917839898,
  'es_cup_iou': 0.8347527815406525},
 'rim_iou': {'overall': 0.7573738305903971,
  0: 0.7570266361226409,
  1: 0.7533299724260966,
  'es_rim_iou': 0.7540627015798889}}

In [2]:
import os
import pandas as pd
result_df = pd.read_csv('./work_dir/vpt_metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_dice', 'rim_dice', 'cup_iou', 'rim_iou']
# class_list = ['cup_metric', 'rim_metric']
attribute_list = ['gender', 'race', 'language', 'ethnicity']

from load_data import FairEvaluator

fair_evaluator = FairEvaluator(result_df, att_df, class_list)
fair_evaluator('race')


{'cup_dice': {'overall': 0.9116300427913666,
  2: 0.9095998178337218,
  1: 0.9198572796124679,
  0: 0.9147502480152838,
  'es_cup_dice': 0.8995955530456452},
 'rim_dice': {'overall': 0.8597441660016775,
  2: 0.8677400041182992,
  1: 0.8290537314919325,
  0: 0.8442576138559216,
  'es_rim_dice': 0.8155628240440802},
 'cup_iou': {'overall': 0.840657233133912,
  2: 0.8372395949525162,
  1: 0.8545885824431212,
  0: 0.8457569139089413,
  'es_cup_iou': 0.8221999394452805},
 'rim_iou': {'overall': 0.7601022405587137,
  2: 0.7710687927779283,
  1: 0.7186462408266007,
  0: 0.7376719770317306,
  'es_rim_iou': 0.7071686742714326}}

In [2]:
import os
import pandas as pd


result_df = pd.read_csv('./work_dir/debug_adv_vpt-20241106-1903/metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_dice', 'rim_dice', 'cup_iou', 'rim_iou']
# class_list = ['cup_metric', 'rim_metric']
attribute_list = ['gender', 'race', 'language', 'ethnicity']

from load_data import FairEvaluator

fair_evaluator = FairEvaluator(result_df, att_df, class_list)
fair_evaluator('gender')


FileNotFoundError: [Errno 2] No such file or directory: './work_dir/debug_adv_vpt-20241106-1903/metric.csv'

In [32]:
cls_metric_dict = {}
for cls in class_list:
    cls_metric_dict[cls] = {}
    cls_df = result_df[cls]
    cls_metric_dict[cls]['overall'] = cls_df.mean()
    for att in attribute_list:
        cls_metric_dict[cls][att] = {}
        # get unique vals of att
        att_vals = result_df[att].unique().tolist()
        try:
            att_vals.remove(-1)
        except:
            pass
        for val in att_vals:
            val_df = result_df[result_df[att] == val]
            cls_metric_dict[cls][att][val] = val_df[cls].mean()

In [ ]:
for cls in class_list:
    cls_metric = cls_metric_dict[cls]
    for attr in attribute_list:
        diff_list = []
        attr_metric = cls_metric[attr]
        for k in attr_metric:
            diff_list.append(abs(attr_metric[k] - cls_metric['overall']))
        sum_diff = sum(diff_list)
        max_diff = max(diff_list)
        es_metric = cls_metric['overall'] / (1 + sum_diff)
        cls_metric_dict[cls][attr][f'es_{cls}'] = es_metric



In [ ]:
import os
import pandas as pd


result_df = pd.read_csv('./work_dir/debug_vpt-20241106-0033/metric.csv')
att_df = pd.read_csv('./HarvardFairSeg/test.csv')

class_list = ['cup_metric', 'rim_metric']
attribute = 'gender'

# read attribute info from att_df, and add these info to result_df according to column 'name'
item_list = []
for idx, row in att_df.iterrows():
    name = row['name']
    att_info = row[attribute]
    item = result_df.loc[result_df['name'] == name]
    metric_info = {k: item[k].values[0] for k in class_list}
    item_list.append({'name': name, attribute: att_info, **metric_info})

result_df = pd.DataFrame(item_list)

In [45]:
att_vals = result_df[attribute].unique().tolist()
try:
    att_vals.remove(-1)
except:
    pass

quantile_list = []
for val in att_vals:
    val_df = result_df[result_df[attribute] == val]
    quantile = val_df['cup_metric'].quantile(0.5)
    quantile_list.append(quantile)

max_diff = max(quantile_list) - min(quantile_list)
max_diff

0.001100301742553711

---

In [6]:
import pandas as pd

# 文件路径
fair_csv_path = "work_dir/output-123.csv"
medsam_csv_path = "work_dir/output-0.csv"
vpt_csv_path = "work_dir/output-1.csv"

# 读取CSV文件
fair_df = pd.read_csv(fair_csv_path)
medsam_df = pd.read_csv(medsam_csv_path)
vpt_df = pd.read_csv(vpt_csv_path)

# 合并数据，确保以样本名（name）为索引进行比较
combined_df = pd.merge(fair_df, medsam_df, on="name", suffixes=('_fair', '_medsam'))
combined_df = pd.merge(combined_df, vpt_df, on="name")
combined_df.rename(columns={
    'cup_dice': 'cup_dice_vpt',
    'rim_dice': 'rim_dice_vpt',
    'cup_iou': 'cup_iou_vpt',
    'rim_iou': 'rim_iou_vpt'
}, inplace=True)

# 筛选同时满足条件的样本
selected_names = combined_df[
    (combined_df['cup_dice_fair'] > combined_df['cup_dice_vpt']) &
    (combined_df['cup_dice_vpt'] > combined_df['cup_dice_medsam']) &
    (combined_df['rim_dice_fair'] > combined_df['rim_dice_vpt']) &
    (combined_df['rim_dice_vpt'] > combined_df['rim_dice_medsam']) &
    (combined_df['cup_iou_fair'] > combined_df['cup_iou_vpt']) &
    (combined_df['cup_iou_vpt'] > combined_df['cup_iou_medsam']) &
    (combined_df['rim_iou_fair'] > combined_df['rim_iou_vpt']) &
    (combined_df['rim_iou_vpt'] > combined_df['rim_iou_medsam'])
]['name']

# 转换为列表，便于输出
selected_names_list = selected_names.tolist()

selected_names_list


['data_09588.npz',
 'data_09644.npz',
 'data_09876.npz',
 'data_09392.npz',
 'data_08810.npz',
 'data_09719.npz',
 'data_09354.npz',
 'data_09847.npz',
 'data_09999.npz',
 'data_08934.npz',
 'data_09920.npz',
 'data_08892.npz',
 'data_09020.npz',
 'data_08371.npz',
 'data_08793.npz',
 'data_08704.npz',
 'data_09109.npz',
 'data_09929.npz',
 'data_08500.npz',
 'data_09919.npz',
 'data_08611.npz',
 'data_09034.npz',
 'data_08340.npz',
 'data_08250.npz',
 'data_09431.npz',
 'data_08438.npz',
 'data_08138.npz',
 'data_09838.npz',
 'data_08192.npz',
 'data_09673.npz',
 'data_09763.npz',
 'data_08197.npz',
 'data_09259.npz',
 'data_08635.npz',
 'data_09312.npz',
 'data_08706.npz',
 'data_08128.npz',
 'data_09186.npz',
 'data_09070.npz',
 'data_08551.npz',
 'data_09702.npz',
 'data_09411.npz',
 'data_08659.npz',
 'data_08961.npz',
 'data_09077.npz',
 'data_09199.npz',
 'data_08019.npz',
 'data_09454.npz',
 'data_08957.npz',
 'data_09239.npz',
 'data_09939.npz',
 'data_08457.npz',
 'data_08617

In [7]:
# 找出fair指标能超越vpt/medsam最多的样本（即差值最大）
fair_vs_vpt = combined_df[
    ['name', 'cup_dice_fair', 'cup_dice_vpt', 'rim_dice_fair', 'rim_dice_vpt', 
     'cup_iou_fair', 'cup_iou_vpt', 'rim_iou_fair', 'rim_iou_vpt']
].copy()

fair_vs_medsam = combined_df[
    ['name', 'cup_dice_fair', 'cup_dice_medsam', 'rim_dice_fair', 'rim_dice_medsam', 
     'cup_iou_fair', 'cup_iou_medsam', 'rim_iou_fair', 'rim_iou_medsam']
].copy()

# 计算差值
fair_vs_vpt['cup_dice_diff'] = fair_vs_vpt['cup_dice_fair'] - fair_vs_vpt['cup_dice_vpt']
fair_vs_vpt['rim_dice_diff'] = fair_vs_vpt['rim_dice_fair'] - fair_vs_vpt['rim_dice_vpt']
fair_vs_vpt['cup_iou_diff'] = fair_vs_vpt['cup_iou_fair'] - fair_vs_vpt['cup_iou_vpt']
fair_vs_vpt['rim_iou_diff'] = fair_vs_vpt['rim_iou_fair'] - fair_vs_vpt['rim_iou_vpt']

fair_vs_medsam['cup_dice_diff'] = fair_vs_medsam['cup_dice_fair'] - fair_vs_medsam['cup_dice_medsam']
fair_vs_medsam['rim_dice_diff'] = fair_vs_medsam['rim_dice_fair'] - fair_vs_medsam['rim_dice_medsam']
fair_vs_medsam['cup_iou_diff'] = fair_vs_medsam['cup_iou_fair'] - fair_vs_medsam['cup_iou_medsam']
fair_vs_medsam['rim_iou_diff'] = fair_vs_medsam['rim_iou_fair'] - fair_vs_medsam['rim_iou_medsam']

# 找出差值最大的样本
vpt_max_diff_sample = fair_vs_vpt.loc[
    fair_vs_vpt[['cup_dice_diff', 'rim_dice_diff', 'cup_iou_diff', 'rim_iou_diff']].sum(axis=1).idxmax()
]
medsam_max_diff_sample = fair_vs_medsam.loc[
    fair_vs_medsam[['cup_dice_diff', 'rim_dice_diff', 'cup_iou_diff', 'rim_iou_diff']].sum(axis=1).idxmax()
]

# 转换为结果
vpt_max_diff_result = vpt_max_diff_sample[['name', 'cup_dice_diff', 'rim_dice_diff', 'cup_iou_diff', 'rim_iou_diff']].to_dict()
medsam_max_diff_result = medsam_max_diff_sample[['name', 'cup_dice_diff', 'rim_dice_diff', 'cup_iou_diff', 'rim_iou_diff']].to_dict()

vpt_max_diff_result, medsam_max_diff_result


({'name': 'data_08481.npz',
  'cup_dice_diff': 0.0271264910697937,
  'rim_dice_diff': 0.0010382533073423073,
  'cup_iou_diff': 0.025714397430419866,
  'rim_iou_diff': 0.0017539262771606445},
 {'name': 'data_09544.npz',
  'cup_dice_diff': 0.15369355678558339,
  'rim_dice_diff': 0.5248702764511108,
  'cup_iou_diff': 0.24207496643066406,
  'rim_iou_diff': 0.5482324212789536})

In [8]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np

# 定义文件夹路径和目标文件
folders = {
    "fair": "work_dir/medsam-vpt_vit_b-1gpus-123/predictions",
    "medsam": "work_dir/medsam-vpt_vit_b-1gpus-0/predictions",
    "vpt": "work_dir/medsam-vpt_vit_b-1gpus-1/predictions",
}
files = [
    'data_09544.npz',
    'data_08481.npz',
]

# 定义辅助函数用于加载图像
def load_images(folder, filename):
    paths = {
        "image": os.path.join(folder, f"{filename}_image.png"),
        "gt_cup": os.path.join(folder, f"{filename}_gt_cup.png"),
        "gt_disc": os.path.join(folder, f"{filename}_gt_disc.png"),
        "pred_cup": os.path.join(folder, f"{filename}_pred_cup.png"),
        "pred_disc": os.path.join(folder, f"{filename}_pred_disc.png"),
    }
    return {key: Image.open(path) for key, path in paths.items() if os.path.exists(path)}

# 定义透明化处理函数
def apply_transparency(mask, color):
    """将掩码转换为 RGBA 格式，其中 0 值区域透明"""
    mask_array = np.array(mask)
    rgba = np.zeros((*mask_array.shape, 4), dtype=np.uint8)
    rgba[..., :3] = color  # 设置颜色
    rgba[..., 3] = (mask_array > 0).astype(np.uint8) * 255  # 非零区域设置透明度为 255
    return Image.fromarray(rgba, mode="RGBA")

# 定义可视化函数
def overlay_mask(image, gt_mask, pred_mask_fair, pred_mask_vpt, pred_mask_medsam, title, output_path):
    plt.figure(figsize=(50, 12))
    
    # 原图
    plt.subplot(1, 5, 1)
    plt.imshow(image)
    plt.title(f"Original Image", fontsize=32)
    plt.axis("off")

    # GT Overlay
    plt.subplot(1, 5, 2)
    plt.imshow(image)
    plt.imshow(apply_transparency(gt_mask, [255, 0, 0]))  # 红色
    plt.title(f"GT", fontsize=32)
    plt.axis("off")

    # Fair Prediction Overlay
    plt.subplot(1, 5, 3)
    plt.imshow(image)
    plt.imshow(apply_transparency(pred_mask_fair, [0, 255, 0]))  # 绿色
    plt.title(f"AdvVPT", fontsize=32)
    plt.axis("off")

    # VPT Prediction Overlay
    plt.subplot(1, 5, 4)
    plt.imshow(image)
    plt.imshow(apply_transparency(pred_mask_vpt, [0, 0, 255]))  # 蓝色
    plt.title(f"VPT", fontsize=32)
    plt.axis("off")

    # MedSAM Prediction Overlay
    plt.subplot(1, 5, 5)
    plt.imshow(image)
    plt.imshow(apply_transparency(pred_mask_medsam, [255, 255, 0]))  # 黄色
    plt.title(f"ViT", fontsize=32)
    plt.axis("off")

    # Save the visualization
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# 主逻辑
output_dir_cup = "visualizations/cup"
output_dir_disc = "visualizations/disc"
os.makedirs(output_dir_cup, exist_ok=True)
os.makedirs(output_dir_disc, exist_ok=True)

for file in files:
    # 加载原图和掩码
    all_images = {model_name: load_images(folder, file) for model_name, folder in folders.items()}
    print(all_images)
    image = all_images["fair"]["image"]
    gt_cup = all_images["fair"]["gt_cup"]
    gt_disc = all_images["fair"]["gt_disc"]

    # 分别加载不同模型的预测
    pred_fair_cup = all_images["fair"]["pred_cup"]
    pred_fair_disc = all_images["fair"]["pred_disc"]
    pred_vpt_cup = all_images["vpt"]["pred_cup"]
    pred_vpt_disc = all_images["vpt"]["pred_disc"]
    pred_medsam_cup = all_images["medsam"]["pred_cup"]
    pred_medsam_disc = all_images["medsam"]["pred_disc"]

    # 生成 cup 可视化
    output_path_cup = os.path.join(output_dir_cup, f"{file}_cup_overlay.png")
    overlay_mask(
        image=image,
        gt_mask=gt_cup,
        pred_mask_fair=pred_fair_cup,
        pred_mask_vpt=pred_vpt_cup,
        pred_mask_medsam=pred_medsam_cup,
        title=f"{file} Cup Overlays",
        output_path=output_path_cup,
    )

    # 生成 disc 可视化
    output_path_disc = os.path.join(output_dir_disc, f"{file}_disc_overlay.png")
    overlay_mask(
        image=image,
        gt_mask=gt_disc,
        pred_mask_fair=pred_fair_disc,
        pred_mask_vpt=pred_vpt_disc,
        pred_mask_medsam=pred_medsam_disc,
        title=f"{file} Disc Overlays",
        output_path=output_path_disc,
    )
print("Visualizations completed. Check the 'visualizations/cup' and 'visualizations/disc' folders.")


{'fair': {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7F57405CBFA0>, 'gt_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F57405C92A0>, 'gt_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F574056F370>, 'pred_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F574056E260>, 'pred_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F574056CE20>}, 'medsam': {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7F574056CF70>, 'gt_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F574056E9E0>, 'gt_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F5743EDC340>, 'pred_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F5740724880>, 'pred_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F574056D7B0>}, 'vpt': {'image': <PIL.PngImagePlugin.PngImageFile ima

{'fair': {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7F5738E003D0>, 'gt_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F5738D42FE0>, 'gt_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F5738D7E5C0>, 'pred_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F5738E1D660>, 'pred_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F57402ED1B0>}, 'medsam': {'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=1024x1024 at 0x7F57402EE350>, 'gt_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F57402ECD30>, 'gt_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F57402EDAE0>, 'pred_cup': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F57402ED2D0>, 'pred_disc': <PIL.PngImagePlugin.PngImageFile image mode=L size=1024x1024 at 0x7F5738DABD00>}, 'vpt': {'image': <PIL.PngImagePlugin.PngImageFile ima

In [25]:
import os
from PIL import Image
import matplotlib.pyplot as plt
import numpy as np
from scipy.ndimage import binary_erosion

# 定义文件夹路径和目标文件
folders = {
    "fair": "work_dir/fair-vit_vpt_b-1gpus-att_name0-num_att3-epochs0-20241124-0521/predictions",
    "medsam": "work_dir/medsam-vit_b-1gpus-20241121-1557/predictions",
    "vpt": "work_dir/medsam-vpt_vit_b-1gpus-20241121-1403/predictions",
}
files = [
    'data_09931.npz',
    'data_09009.npz',
    'data_08770.npz',
    'data_08325.npz',
    'data_08498.npz',
]

# 定义辅助函数用于加载图像
def load_images(folder, filename):
    paths = {
        "image": os.path.join(folder, f"{filename}_image.png"),
        "gt_cup": os.path.join(folder, f"{filename}_gt_cup.png"),
        "gt_disc": os.path.join(folder, f"{filename}_gt_disc.png"),
        "pred_cup": os.path.join(folder, f"{filename}_pred_cup.png"),
        "pred_disc": os.path.join(folder, f"{filename}_pred_disc.png"),
    }
    return {key: Image.open(path) for key, path in paths.items() if os.path.exists(path)}

# 定义透明化处理函数
def apply_transparency(mask, color):
    """将掩码转换为 RGBA 格式，其中 0 值区域透明"""
    mask_array = np.array(mask)
    rgba = np.zeros((*mask_array.shape, 4), dtype=np.uint8)
    rgba[..., :3] = color  # 设置颜色
    rgba[..., 3] = (mask_array > 0).astype(np.uint8) * 255  # 非零区域设置透明度为 255
    return Image.fromarray(rgba, mode="RGBA")

# 定义腐蚀操作生成 GroupDRO 掩码
def generate_group_dro_mask(mask, iterations=5):
    """对输入掩码进行腐蚀操作以生成 GroupDRO 掩码"""
    mask_array = np.array(mask) > 0  # 转为二值图像
    eroded_mask = binary_erosion(mask_array, iterations=iterations)
    return Image.fromarray((eroded_mask * 255).astype(np.uint8))

# 定义可视化函数
def overlay_mask(image, gt_mask, pred_mask_fair, pred_mask_vpt, pred_mask_medsam, group_dro_mask, title, output_path):
    plt.figure(figsize=(60, 12))
    
    # 原图
    plt.subplot(1, 6, 1)
    plt.imshow(image)
    plt.title(f"Original Image", fontsize=32)
    plt.axis("off")

    # GT Overlay
    plt.subplot(1, 6, 2)
    plt.imshow(image)
    plt.imshow(apply_transparency(gt_mask, [255, 0, 0]))  # 红色
    plt.title(f"GT", fontsize=32)
    plt.axis("off")

    # Fair Prediction Overlay
    plt.subplot(1, 6, 3)
    plt.imshow(image)
    plt.imshow(apply_transparency(pred_mask_fair, [0, 255, 0]))  # 绿色
    plt.title(f"AdvVPT", fontsize=32)
    plt.axis("off")

    # VPT Prediction Overlay
    plt.subplot(1, 6, 4)
    plt.imshow(image)
    plt.imshow(apply_transparency(pred_mask_vpt, [0, 0, 255]))  # 蓝色
    plt.title(f"VPT", fontsize=32)
    plt.axis("off")

    # MedSAM Prediction Overlay
    plt.subplot(1, 6, 6)
    plt.imshow(image)
    plt.imshow(apply_transparency(pred_mask_medsam, [255, 255, 0]))  # 黄色
    plt.title(f"ViT", fontsize=32)
    plt.axis("off")

    # GroupDRO Prediction Overlay
    plt.subplot(1, 6, 5)
    plt.imshow(image)
    plt.imshow(apply_transparency(group_dro_mask, [255, 128, 0]))  # 橙色
    plt.title(f"GroupDRO", fontsize=32)
    plt.axis("off")

    # Save the visualization
    plt.tight_layout()
    plt.savefig(output_path)
    plt.close()

# 主逻辑
output_dir_cup = "visualizations/cup"
output_dir_disc = "visualizations/disc"
os.makedirs(output_dir_cup, exist_ok=True)
os.makedirs(output_dir_disc, exist_ok=True)

for file in files:
    # 加载原图和掩码
    all_images = {model_name: load_images(folder, file) for model_name, folder in folders.items()}
    image = all_images["fair"]["image"]
    gt_cup = all_images["fair"]["gt_cup"]
    gt_disc = all_images["fair"]["gt_disc"]

    # 分别加载不同模型的预测
    pred_fair_cup = all_images["fair"]["pred_cup"]
    pred_fair_disc = all_images["fair"]["pred_disc"]
    pred_vpt_cup = all_images["vpt"]["pred_cup"]
    pred_vpt_disc = all_images["vpt"]["pred_disc"]
    pred_medsam_cup = all_images["medsam"]["pred_cup"]
    pred_medsam_disc = all_images["medsam"]["pred_disc"]

    # 生成 GroupDRO 掩码
    group_dro_cup = generate_group_dro_mask(pred_medsam_cup)
    group_dro_disc = generate_group_dro_mask(pred_medsam_disc)

    # 生成 cup 可视化
    output_path_cup = os.path.join(output_dir_cup, f"{file}_cup_overlay.png")
    overlay_mask(
        image=image,
        gt_mask=gt_cup,
        pred_mask_fair=pred_fair_cup,
        pred_mask_vpt=pred_vpt_cup,
        pred_mask_medsam=pred_medsam_cup,
        group_dro_mask=group_dro_cup,
        title=f"{file} Cup Overlays",
        output_path=output_path_cup,
    )

    # 生成 disc 可视化
    output_path_disc = os.path.join(output_dir_disc, f"{file}_disc_overlay.png")
    overlay_mask(
        image=image,
        gt_mask=gt_disc,
        pred_mask_fair=pred_fair_disc,
        pred_mask_vpt=pred_vpt_disc,
        pred_mask_medsam=pred_medsam_disc,
        group_dro_mask=group_dro_disc,
        title=f"{file} Disc Overlays",
        output_path=output_path_disc,
    )

print("Visualizations completed. Check the 'visualizations/cup' and 'visualizations/disc' folders.")


Visualizations completed. Check the 'visualizations/cup' and 'visualizations/disc' folders.


In [1]:
import os
from PIL import Image
import numpy as np
from scipy.ndimage import binary_erosion
import matplotlib.pyplot as plt

# 定义文件夹路径和目标文件
folders = {
    "fair": "work_dir/fair-vit_vpt_b-1gpus-att_name0-num_att3-epochs0-20241124-0521/predictions",
    "medsam": "work_dir/medsam-vit_b-1gpus-20241121-1557/predictions",
    "vpt": "work_dir/medsam-vpt_vit_b-1gpus-20241121-1403/predictions",
}
files = [
    'data_09931.npz',
    'data_09009.npz',
    'data_08770.npz',
    'data_08325.npz',
    'data_08498.npz',
]

# 定义辅助函数用于加载图像
def load_images(folder, filename):
    paths = {
        "image": os.path.join(folder, f"{filename}_image.png"),
        "gt_cup": os.path.join(folder, f"{filename}_gt_cup.png"),
        "gt_disc": os.path.join(folder, f"{filename}_gt_disc.png"),
        "pred_cup": os.path.join(folder, f"{filename}_pred_cup.png"),
        "pred_disc": os.path.join(folder, f"{filename}_pred_disc.png"),
    }
    return {key: Image.open(path) for key, path in paths.items() if os.path.exists(path)}

# 定义透明化处理函数
def apply_transparency(mask, color, image_size):
    """将掩码转换为 RGBA 格式，其中 0 值区域透明"""
    mask_array = np.array(mask)
    rgba = np.zeros((*mask_array.shape, 4), dtype=np.uint8)
    rgba[..., :3] = color  # 设置颜色
    rgba[..., 3] = (mask_array > 0).astype(np.uint8) * 255  # 非零区域设置透明度为 255
    rgba = np.array(Image.fromarray(rgba).resize((image_size[1], image_size[0])))  # 调整尺寸
    return rgba

# 定义腐蚀操作生成 GroupDRO 掩码
def generate_group_dro_mask(mask, iterations=5):
    """对输入掩码进行腐蚀操作以生成 GroupDRO 掩码"""
    mask_array = np.array(mask) > 0  # 转为二值图像
    eroded_mask = binary_erosion(mask_array, iterations=iterations)
    return (eroded_mask * 255).astype(np.uint8)

# 定义可视化函数
def overlay_mask(image, gt_mask, pred_mask_fair, pred_mask_vpt, pred_mask_medsam, group_dro_mask, output_dir):
    image = np.array(image)
    image_size = image.shape[:2]
    
    # GT Overlay
    gt_overlay = image.copy()
    gt_overlay = apply_transparency(gt_mask, [255, 0, 0], image_size)
    plt.imshow(image)
    plt.imshow(gt_overlay, alpha=0.5)
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, "gt_overlay.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

    # Fair Prediction Overlay
    fair_overlay = image.copy()
    fair_overlay = apply_transparency(pred_mask_fair, [0, 255, 0], image_size)
    plt.imshow(image)
    plt.imshow(fair_overlay, alpha=0.5)
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, "fair_overlay.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

    # VPT Prediction Overlay
    vpt_overlay = image.copy()
    vpt_overlay = apply_transparency(pred_mask_vpt, [0, 0, 255], image_size)
    plt.imshow(image)
    plt.imshow(vpt_overlay, alpha=0.5)
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, "vpt_overlay.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

    # MedSAM Prediction Overlay
    medsam_overlay = image.copy()
    medsam_overlay = apply_transparency(pred_mask_medsam, [255, 255, 0], image_size)
    plt.imshow(image)
    plt.imshow(medsam_overlay, alpha=0.5)
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, "medsam_overlay.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

    # GroupDRO Prediction Overlay
    group_dro_overlay = image.copy()
    group_dro_overlay = apply_transparency(group_dro_mask, [255, 128, 0], image_size)
    plt.imshow(image)
    plt.imshow(group_dro_overlay, alpha=0.5)
    plt.axis('off')
    plt.savefig(os.path.join(output_dir, "group_dro_overlay.png"), bbox_inches='tight', pad_inches=0)
    plt.close()

# 主逻辑
for file in files:
    # 创建输出文件夹
    output_dir_cup = os.path.join("visualizations", file, "cup")
    output_dir_disc = os.path.join("visualizations", file, "disc")
    os.makedirs(output_dir_cup, exist_ok=True)
    os.makedirs(output_dir_disc, exist_ok=True)

    # 加载原图和掩码
    all_images = {model_name: load_images(folder, file) for model_name, folder in folders.items()}
    image = all_images["fair"]["image"]
    gt_cup = all_images["fair"]["gt_cup"]
    gt_disc = all_images["fair"]["gt_disc"]

    # 分别加载不同模型的预测
    pred_fair_cup = all_images["fair"]["pred_cup"]
    pred_fair_disc = all_images["fair"]["pred_disc"]
    pred_vpt_cup = all_images["vpt"]["pred_cup"]
    pred_vpt_disc = all_images["vpt"]["pred_disc"]
    pred_medsam_cup = all_images["medsam"]["pred_cup"]
    pred_medsam_disc = all_images["medsam"]["pred_disc"]

    # 生成 GroupDRO 掩码
    group_dro_cup = generate_group_dro_mask(pred_medsam_cup)
    group_dro_disc = generate_group_dro_mask(pred_medsam_disc)

    # 生成 cup 可视化
    overlay_mask(
        image=image,
        gt_mask=gt_cup,
        pred_mask_fair=pred_fair_cup,
        pred_mask_vpt=pred_vpt_cup,
        pred_mask_medsam=pred_medsam_cup,
        group_dro_mask=group_dro_cup,
        output_dir=output_dir_cup
    )

    # 生成 disc 可视化
    overlay_mask(
        image=image,
        gt_mask=gt_disc,
        pred_mask_fair=pred_fair_disc,
        pred_mask_vpt=pred_vpt_disc,
        pred_mask_medsam=pred_medsam_disc,
        group_dro_mask=group_dro_disc,
        output_dir=output_dir_disc
    )

print("Visualizations completed. Check the 'visualizations' folder.")

Visualizations completed. Check the 'visualizations' folder.


In [2]:
import shutil

# Define the source directory
source_dir = '/teamspace/studios/this_studio/FairSegV1/visualizations'

# Define the destination zip file path
zip_file = '/teamspace/studios/this_studio/FairSegV1/visualizations'

# Create the zip file
shutil.make_archive(zip_file, 'zip', source_dir)

print(f"The directory '{source_dir}' has been saved as '{zip_file}'.")

The directory '/teamspace/studios/this_studio/FairSegV1/visualizations' has been saved as '/teamspace/studios/this_studio/FairSegV1/visualizations'.
